In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from collections import defaultdict, Counter
from pathlib import Path
from typing import Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
import torch
from tqdm.auto import tqdm

from src.analysis.state_space import prepare_state_trajectory, StateSpaceAnalysisSpec, flatten_trajectory
from src.utils.timit import get_word_metadata

In [ ]:
model_dir = "outputs/models/librispeech-train-clean-100/w2v2_8/discrim-rnn_32-mAP1/word_broad_10frames_fixedlen25"
output_dir = "."
dataset_path = "outputs/preprocessed_data/librispeech-train-clean-100"
equivalence_path = "outputs/equivalence_datasets/librispeech-train-clean-100/w2v2_8/word_broad_10frames_fixedlen25/equivalence.pkl"
hidden_states_path = "outputs/hidden_states/librispeech-train-clean-100/w2v2_8/hidden_states.h5"
state_space_specs_path = "outputs/state_space_specs/librispeech-train-clean-100/w2v2_8/state_space_specs.h5"
embeddings_path = "outputs/model_embeddings/librispeech-train-clean-100/w2v2_8/discrim-rnn_32-mAP1/word_broad_10frames_fixedlen25/librispeech-train-clean-100.npy"

model_dir = "outputs/models/librispeech-train-clean-100/w2v2_8/discrim-rnn_32-mAP2/word_broad_10frames"
output_dir = "."
dataset_path = "outputs/preprocessed_data/librispeech-train-clean-100"
equivalence_path = "outputs/equivalence_datasets/librispeech-train-clean-100/w2v2_8/word_broad_10frames/equivalence.pkl"
hidden_states_path = "outputs/hidden_states/librispeech-train-clean-100/w2v2_8/hidden_states.h5"
state_space_specs_path = "outputs/state_space_specs/librispeech-train-clean-100/w2v2_8/state_space_specs.h5"
embeddings_path = "outputs/model_embeddings/librispeech-train-clean-100/w2v2_8/discrim-rnn_32-mAP2/word_broad_10frames/librispeech-train-clean-100.npy"

metric = "cosine"

# Retain words with N or more instances
retain_n = 10

subsample_instances = 50

model_sfreq = 50

expand_window = (15, 0)

In [ ]:
with open(embeddings_path, "rb") as f:
    model_representations: np.ndarray = np.load(f)
state_space_spec = StateSpaceAnalysisSpec.from_hdf5(state_space_specs_path, "word")
assert state_space_spec.is_compatible_with(model_representations)

In [ ]:
label_counts = state_space_spec.label_counts
drop_labels = label_counts[label_counts < retain_n].index
state_space_spec = state_space_spec.drop_labels(drop_names=drop_labels)

In [ ]:
state_space_spec = state_space_spec.subsample_instances(subsample_instances)

In [ ]:
metadata = get_word_metadata(state_space_spec)

In [ ]:
trajectory = prepare_state_trajectory(model_representations, state_space_spec, pad=np.nan,
                                      expand_window=expand_window)
lengths = [np.isnan(traj_i[:, :, 0]).argmax(axis=1) for traj_i in trajectory]

In [ ]:
traj_full, traj_full_flat_src = flatten_trajectory(trajectory)

In [ ]:
len(trajectory), np.concatenate(lengths).mean()

In [ ]:
pca_full = PCA(n_components=4)
traj_full_flat_pca = pca_full.fit_transform(traj_full)

#### Prepare truncated trajectory data

In [ ]:
traj_trunc = [traj_i[:, :(expand_window[0] + 15)] for traj_i in trajectory]
traj_trunc_flat, traj_trunc_flat_src = flatten_trajectory(traj_trunc)

In [ ]:
trunc_times = np.concatenate([-np.arange(expand_window[0], -1, -1), np.arange(1, 15)])

In [ ]:
pca = PCA(n_components=4)
traj_trunc_flat_pca = pca.fit_transform(traj_trunc_flat)

## State space

In [ ]:
traj_full_flat_src_dict = {src: i for i, src in enumerate(map(tuple, traj_full_flat_src))}

In [ ]:
traj_full_flat_frame_dict = defaultdict(list)
for i, src in enumerate(traj_full_flat_src):
    traj_full_flat_frame_dict[tuple(src[:2])].append(i)

In [ ]:
len(traj_full_flat_frame_dict[6, 0])

In [ ]:
def plot_state_space_binned(n_bins, groupby=None, return_data=False):
    all_lengths = np.concatenate(lengths)
    max_traj_length = all_lengths.max()

    # bin word tokens by length
    length_bins = pd.qcut(all_lengths, q=n_bins, labels=np.arange(n_bins), retbins=True)[1]
    bin_time_edges = np.maximum(0, length_bins) / model_sfreq
    bin_assignments = [np.digitize(traj_lengths, length_bins, right=True) - 1
                       for traj_lengths in lengths]    
    all_bin_assignments = np.concatenate(bin_assignments)
    all_bin_assignments = all_bin_assignments[all_bin_assignments >= 0]

    # key := bin + grouping variables
    # build reverse map from key and frame index to list of (label_idx, instance_idx) tuples
    bin_assignments_rev = defaultdict(list)
    if groupby is not None:
        group_lookup = metadata[groupby].to_dict()
    for label_idx, assignments_i in enumerate(bin_assignments):
        for j, bin_idx in enumerate(assignments_i):
            if bin_idx < 0:
                continue

            if groupby is not None:
                group_value = group_lookup[state_space_spec.labels[label_idx], j]
                if group_value is None or (isinstance(group_value, float) and np.isnan(group_value)):
                    continue
                key = (bin_idx, group_value)
            else:
                key = (bin_idx,)
            bin_assignments_rev[key].append((label_idx, j))

    # Prepare per-key vector collections
    if return_data:
        bin_frame_data, bin_frame_src = {}, {}
        for key, traj_indices in bin_assignments_rev.items():
            data_i, src_i = [], []
            for label_idx, instance_idx in traj_indices:
                flat_idxs = traj_full_flat_frame_dict[label_idx, instance_idx]
                data_i.append(traj_full_flat_pca[flat_idxs])
                src_i.append(traj_full_flat_src[flat_idxs])

            bin_frame_data[key] = data_i
            bin_frame_src[key] = src_i

    # Prepare per-frame and per-key means
    bin_frame_means = defaultdict(list)
    for key, traj_indices in tqdm(bin_assignments_rev.items(), desc="retrieving per-bin data"):
        for frame_idx in range(max_traj_length):
            flat_idxs = [traj_full_flat_src_dict[(label_idx, instance_idx, frame_idx)]
                        for label_idx, instance_idx in traj_indices
                        if (label_idx, instance_idx, frame_idx) in traj_full_flat_src_dict]
            
            if len(flat_idxs) == 0:
                bin_frame_means[key].append(np.full(traj_full_flat_pca.shape[1], np.nan))
            else:
                bin_frame_means[key].append(traj_full_flat_pca[flat_idxs].mean(axis=0))

    bin_frame_means = {key: np.array(frame_means_i) for key, frame_means_i in bin_frame_means.items()}

    ## Plot
    pcs = [[0, 1], [2, 3]]
    f, axs = plt.subplots(1, len(pcs), figsize=(10 * len(pcs), 10), squeeze=False)

    # bin_colors = sns.color_palette("tab10", n_bins)
    # get normalized continuous hue for bin edges
    bin_colors = sns.color_palette("spring", n_bins)
    grouping_values = sorted(set(key[1:] for key in bin_frame_means.keys()))
    grouping_styles = ["-", "--", "-.", ":", (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5, 1, 5))]
    assert len(grouping_values) <= len(grouping_styles)

    for i, (pcs_i, ax) in enumerate(zip(pcs, axs.flat)):
        ax.axvline(0, color="black", linestyle="--")
        ax.axhline(0, color="black", linestyle="--")

        i, j = pcs_i
        ax.set_xlabel(f"PC {i+1}")
        ax.set_ylabel(f"PC {j+1}")

        k = 0
        for key in sorted(bin_frame_means.keys()):
            bin_idx = key[0]
            grouping_values_ij = key[1:]
            means = bin_frame_means[key]
            bin_edge = bin_time_edges[bin_idx]
            ax.plot(means[:, i], means[:, j], label=f"{key[1:]} {bin_edge:.2f} s",
                    color=bin_colors[bin_idx],
                    linestyle=grouping_styles[grouping_values.index(grouping_values_ij)],
                    alpha=0.7)

            ax.quiver(means[:-1, i], means[:-1, j], means[1:, i] - means[:-1, i], means[1:, j] - means[:-1, j],
                        angles='xy', scale_units='xy', scale=1, color="gray", alpha=0.5)
            
            # O at start of word
            word_start_frame = 0 + expand_window[0]
            ax.scatter(means[word_start_frame, i], means[word_start_frame, j], color="blue", marker="o")

            # X at middle of word
            if np.isnan(means).any():
                max_length = np.isnan(means).argmax(0).min() - 1
            else:
                max_length = means.shape[0]
            word_midpoint_frame = (max_length - word_start_frame) // 2 + word_start_frame
            ax.scatter(means[word_midpoint_frame, i], means[word_midpoint_frame, j], color="red", marker="x")

        # legend on last axis
        handles, labels = [], []
        from matplotlib import patches as mpatches
        from matplotlib.lines import Line2D
        handles.append(mpatches.Patch(color="white", label=""))
        labels.append("length bin")
        for bin in range(n_bins):
            handles.append(mpatches.Patch(color=bin_colors[bin], label=f"bin {bin}"))
            labels.append(f"{bin_time_edges[bin]:.2f} s")

        if groupby is not None:
            handles.append(mpatches.Patch(color="white", label=""))
            labels.append(groupby)
            for group, style in zip(grouping_values, grouping_styles):
                handles.append(Line2D([0], [0], color="black", linestyle=style))
                labels.append(group)

        ax.legend(handles, labels, loc="upper right", bbox_to_anchor=(1.5, 1))

    if return_data:
        return f, bin_frame_data, bin_frame_src, bin_frame_means, bin_time_edges
    else:
        return f

In [ ]:
f, ss_data_all, ss_src_all, ss_means_all, ss_edges_all = plot_state_space_binned(5, return_data=True)
f.savefig(Path(output_dir) / "state_space.png", bbox_inches="tight")

In [ ]:
cluster_key = (1,)
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10, n_init="auto")
cluster_length = min(x.shape[0] for x in ss_data_all[cluster_key])
print(cluster_length)
cluster_data = np.array([t[:cluster_length] for t in ss_data_all[cluster_key]])
km.fit(cluster_data.reshape(cluster_data.shape[0], -1))

In [ ]:
np.bincount(km.labels_)

In [ ]:
cluster_word_tokens = np.array([state_space_spec.labels[src[0][0]] for src in ss_src_all[cluster_key]])

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 10))
for i, center in enumerate(km.cluster_centers_):
    center = center.reshape(cluster_data.shape[1:])
    ax.plot(center[:, 0], center[:, 1], marker="o", label=str(i))

    # cluster_trajs = cluster_data[km.labels_ == i].reshape(-1, *center.shape)
    # for traj in cluster_trajs[:2]:
    #     ax.plot(traj[:, 0], traj[:, 1], color="gray", alpha=0.5)

    ax.scatter(center[0, 0], center[0, 1], marker="x", s=100)
ax.legend()

In [ ]:
for cluster_i in range(len(km.cluster_centers_)):
    cluster_idxs = np.where(km.labels_ == cluster_i)[0]
    print(f"Cluster {cluster_i}")
    samples = np.random.choice(cluster_idxs, 20, replace=False)
    print(" ".join(cluster_word_tokens[idx] for idx in samples))
    print(Counter(cluster_word_tokens[cluster_idxs]).most_common(5))

In [ ]:
# plot k random traces
k = 100
f, ax = plt.subplots(1, 1, figsize=(5, 5))
plot_key = (2,)
for i in range(k):
    key = np.random.choice(len(ss_data_all[plot_key]))
    data_i = ss_data_all[plot_key][key]
    src_i = ss_src_all[plot_key][key]
    data_i = data_i - data_i[0]
    ax.plot(data_i[:, 0], data_i[:, 1], label=state_space_spec.labels[src_i[0][0]], alpha=0.3)
ax.legend(loc="upper right", bbox_to_anchor=(1.5, 1))

In [ ]:
norm_track_key = (0,)
maxlen = max(data_i.shape[0] for data_i in ss_data_all[norm_track_key])
norm_track_data = np.full((len(ss_data_all[norm_track_key]), maxlen), np.nan)
diff_norm_track_data = np.full((len(ss_data_all[norm_track_key]), maxlen), np.nan)
for i, data_i in enumerate(ss_data_all[norm_track_key]):
    norm_track_data[i, :data_i.shape[0]] = np.linalg.norm(data_i, axis=1)
    diff_norm_track_data[i, :data_i.shape[0]] = np.linalg.norm(np.roll(data_i, -1, axis=0) - data_i, axis=1)

# order by peak time
norm_track_data = norm_track_data[norm_track_data.argmax(axis=1).argsort()]

In [ ]:
plt.plot(np.nanmean(norm_track_data, 0))

In [ ]:
plt.plot(np.nanmean(diff_norm_track_data, 0))

In [ ]:
f, ss_data_stress, ss_src_stress, ss_means_stress, ss_edges_stress = plot_state_space_binned(5, groupby="stress_primary_initial", return_data=True)
f.savefig(Path(output_dir) / "state_space-stress_primary_initial.png", bbox_inches="tight")

In [ ]:
f, ss_data_freq, ss_src_freq, ss_means_freq, ss_edges_freq = plot_state_space_binned(5, groupby="word_frequency_quantile", return_data=True)
f.savefig(Path(output_dir) / "state_space-word_frequency_quantile.png", bbox_inches="tight")

## Basic plots

In [ ]:
pca.explained_variance_ratio_

In [ ]:
traj_idxs_by_frame = []
for frame_idx in range(traj_trunc_flat_src[:, 2].max() + 1):
    traj_idxs_by_frame.append(np.where(traj_trunc_flat_src[:, 2] == frame_idx)[0])

In [ ]:
frame_means, frame_sds, frame_counts = [], [], []
for frame_idx, traj_idxs_i in enumerate(traj_idxs_by_frame):
    frame_means.append(traj_trunc_flat_pca[traj_idxs_i].mean(axis=0))
    frame_sds.append(traj_trunc_flat_pca[traj_idxs_i].std(axis=0))
    frame_counts.append(len(traj_idxs_i))

frame_means = np.array(frame_means)
frame_sds = np.array(frame_sds)
frame_counts = np.array(frame_counts)

In [ ]:
f, ax = plt.subplots()

times = np.arange(-expand_window[0], frame_means.shape[0] - expand_window[0]) / model_sfreq
ax.axvline(0, color="gray", linestyle="--")
ax.axhline(0, color="gray", linestyle="--")

for component in range(frame_means.shape[1]):
    ax.plot(times, frame_means[:, component], label=f"PC {component+1}")
    ax.fill_between(times,
                    frame_means[:, component] - frame_sds[:, component] / np.sqrt(frame_counts),
                    frame_means[:, component] + frame_sds[:, component] / np.sqrt(frame_counts),
                    alpha=0.3)

ax.legend()
ax.set_xlabel("Distance from word boundary")

f.savefig(Path(output_dir) / "word_boundary.png")

### By various grouping variables

In [ ]:
metadata["label_idx"] = metadata.index.get_level_values("label").map({label: idx for idx, label in enumerate(state_space_spec.labels)})

# prepare onset category annotation
categorization = {
    "consonant": "B CH D DH F G HH JH K L M N NG P R S SH T TH V W Y Z ZH".split(" "),
    "vowel": "AA AE AH AO AW AY EH ER EY IH IY OW OY UH UW".split(" "),
}
category_lookup = {label: category for category, labels in categorization.items() for label in labels}
metadata["onset_phoneme_category"] = metadata.onset_phoneme.map(category_lookup)

In [ ]:
traj_trunc_flat_src_dict = {src: i for i, src in enumerate(map(tuple, traj_trunc_flat_src))}

In [ ]:
def plot_boundary_grouped(grouping_variable: Optional[str], num_components=4):
    grouped_frame_means, grouped_frame_sds, grouped_frame_counts = {}, {}, {}

    metadata_ = metadata.reset_index().set_index(["label_idx", "instance_idx"])
    if grouping_variable is None:
        grouper = [(None, metadata_)]
    else:
        grouper = metadata_.groupby(grouping_variable)
    for group_values, rows in tqdm(grouper):
        matched_word_instances = rows.index
        
        frame_means_i, frame_sds_i, frame_counts_i = [], [], []
        for frame_idx in range(traj_trunc_flat_src[:, 2].max() + 1):
            traj_idxs_i = [traj_trunc_flat_src_dict[(label_idx, instance_idx, frame_idx)]
                           for label_idx, instance_idx in matched_word_instances
                           if (label_idx, instance_idx, frame_idx) in traj_trunc_flat_src_dict]
            if len(traj_idxs_i) == 0:
                frame_means_i.append(np.full(traj_trunc_flat_pca.shape[1], np.nan))
                frame_sds_i.append(np.full(traj_trunc_flat_pca.shape[1], np.nan))
                frame_counts_i.append(0)
            else:
                frame_means_i.append(traj_trunc_flat_pca[traj_idxs_i].mean(axis=0))
                frame_sds_i.append(traj_trunc_flat_pca[traj_idxs_i].std(axis=0))
                frame_counts_i.append(len(traj_idxs_i))

        grouped_frame_means[group_values] = np.array(frame_means_i)
        grouped_frame_sds[group_values] = np.array(frame_sds_i)
        grouped_frame_counts[group_values] = np.array(frame_counts_i)

    f, ax = plt.subplots()

    times = np.arange(-expand_window[0], frame_means.shape[0] - expand_window[0]) / model_sfreq
    ax.axvline(0, color="gray", linestyle="--")
    ax.axhline(0, color="gray", linestyle="--")

    # set ylimits to 10%, 90% percentile activation
    yl, yu = np.nanpercentile(traj_trunc_flat_pca, (10, 90), axis=0)
    ax.set_ylim(yl.min(), yu.max())

    component_palette = sns.color_palette("tab10", frame_means.shape[1])
    group_palette = ["-", "--", ":"]
    for i, group in enumerate(grouped_frame_means):
        for component in range(min(num_components, frame_means.shape[1])):
            ax.plot(times, grouped_frame_means[group][:, component],
                    linestyle=group_palette[i], color=component_palette[component],
                    label=f"{group}-{component}")
            ax.fill_between(times,
                            grouped_frame_means[group][:, component] - grouped_frame_sds[group][:, component] / np.sqrt(grouped_frame_counts[group]),
                            grouped_frame_means[group][:, component] + grouped_frame_sds[group][:, component] / np.sqrt(grouped_frame_counts[group]),
                            color=component_palette[component], alpha=0.3)

    handles, labels = [], []
    from matplotlib import patches as mpatches
    from matplotlib.lines import Line2D
    handles.append(mpatches.Patch(color="white", label=""))
    labels.append("component")
    for component in range(min(num_components, frame_means.shape[1])):
        handles.append(mpatches.Patch(color=component_palette[component], label=str(component + 1)))
        labels.append(str(component + 1))

    if grouping_variable is not None:
        handles.append(mpatches.Patch(color="white", label=""))
        labels.append(grouping_variable)
        for i, group in enumerate(grouped_frame_means):
            handles.append(Line2D([0], [0], color="black", linestyle=group_palette[i]))
            labels.append(group)

    ax.legend(handles, labels, loc="upper right", bbox_to_anchor=(1.5, 1))

    return f, ax

In [ ]:
for grouping_variable in [None, "stress_primary_initial", "word_frequency_quantile", "onset_phoneme_category"]:
    f, ax = plot_boundary_grouped(grouping_variable)
    ax.set_title(grouping_variable)

    if grouping_variable is not None:
        fname = f"word_boundary-{grouping_variable}.png"
    else:
        fname = "word_boundary.png"
    f.savefig(Path(output_dir) / fname, bbox_inches="tight")

## Cluster trajs

In [ ]:
# traj_cluster_trunc = [traj_i[:, cluster_sample_idxs] for traj_i in traj_trunc]
# traj_cluster_flat, traj_cluster_src = flatten_trajectory(traj_cluster_trunc)
traj_cluster_trunc = traj_trunc
traj_cluster_flat, traj_cluster_src = traj_trunc_flat, traj_trunc_flat_src

In [ ]:
# pca_cluster = PCA(n_components=4)
# traj_cluster_flat_pca = pca_cluster.fit_transform(traj_cluster_flat)
pca_cluster = pca
traj_cluster_flat_pca = traj_trunc_flat_pca

In [ ]:
# chunk points: wherever 1st or second column of traj_cluster_src changes
chunk_points = np.where(np.any(np.diff(traj_cluster_src[:, :2], axis=0), axis=1))[0] + 1
traj_cluster_word_level = np.split(traj_cluster_flat_pca, chunk_points)

In [ ]:
traj_cluster_word_level_src = np.split(traj_cluster_src, chunk_points)
traj_cluster_word_level_src = np.array([src_i[0][:2] for src_i in traj_cluster_word_level_src])

In [ ]:
maxlen = max(map(len, traj_cluster_word_level))
traj_cluster_mat = np.zeros((len(traj_cluster_word_level), maxlen, traj_cluster_flat_pca.shape[1]))
for i, traj_i in enumerate(tqdm(traj_cluster_word_level)):
    traj_cluster_mat[i, :traj_i.shape[0], :] = traj_i

In [ ]:
# baseline around center
center_frame = traj_cluster_mat.shape[1] // 2
baseline_data = traj_cluster_mat[:, center_frame - 4: center_frame + 4, :].mean(axis=1, keepdims=True)
traj_cluster_mat -= baseline_data

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, n_init="auto")
km.fit(traj_cluster_mat.reshape(traj_cluster_mat.shape[0], -1))

In [ ]:
centroids = km.cluster_centers_.reshape((km.cluster_centers_.shape[0], *traj_cluster_mat.shape[1:]))

# add baseline back in
centroids += baseline_data.mean(0)[-1][None, None, :]

f, axs = plt.subplots(1, centroids.shape[2], figsize=(6 * centroids.shape[2], 4))
for i, (ax, centroid) in enumerate(zip(axs, centroids.transpose(2, 0, 1))):
    ax.axvline(0, color="black", linestyle="--")
    ax.axhline(0, color="black", linestyle="--")
    ax.set_title(f"PCA {i + 1}")
    for j, c in enumerate(centroid):
        ax.plot(trunc_times, c, label=f"cluster {j}")
    ax.legend()

f.tight_layout()

In [ ]:
np.bincount(km.labels_)

In [ ]:
from collections import Counter

match_df = {}
for km_label in range(len(km.cluster_centers_)):
    matches = np.where(km.labels_ == km_label)[0]
    matches_src = traj_cluster_word_level_src[matches]
    matches_labels = [state_space_spec.labels[idx] for idx in matches_src[:, 0]]
    match_df[km_label] = pd.DataFrame(Counter(matches_labels).items(), columns=["label", "count"])
match_df = pd.concat(match_df, names=["cluster"]).droplevel(-1)
cluster_df = pd.merge(match_df.reset_index(), metadata.groupby("label").first().reset_index(), on="label")
cluster_df["word_log_frequency"] = np.log10(cluster_df.word_frequency)

In [ ]:
sns.barplot(data=cluster_df.groupby("cluster")["word_frequency_quantile"].value_counts(normalize=True).reset_index(),
            hue="word_frequency_quantile", x="cluster", y="proportion")

In [ ]:
sns.barplot(data=cluster_df, x="cluster", y="word_log_frequency")

In [ ]:
sns.barplot(data=cluster_df.groupby("cluster")["num_syllables"].value_counts(normalize=True).reset_index(),
            hue="num_syllables", x="cluster", y="proportion")

In [ ]:
cluster_df.groupby("cluster").onset_phoneme.value_counts(normalize=True).to_frame().unstack().T.plot(kind="bar")

In [ ]:
cluster_df.groupby("cluster").stress_primary_initial.value_counts(normalize=True).to_frame().unstack().T.plot(kind="bar")

In [ ]:
cluster_df.groupby("cluster").sample(10)

In [ ]:
# cluster entropy within word type
word_cluster_entropy = cluster_df.groupby("label").cluster.value_counts(normalize=True).groupby("label").apply(lambda x: -np.sum(x * np.log2(x)))
word_cluster_entropy.sort_values().head(20)

In [ ]:
word_cluster_entropy.sort_values().tail(20)